# UASGs
> Neste notebook baixamos os dados de materiais da api disponibilizada pelo MGI (dados abertos) e criamos uma base.
>
> A documentação da API do MGI pode ser encontrada em https://dadosabertos.compras.gov.br/swagger-ui/index.html#/

In [1]:
import json
import duckdb
import ipywidgets
from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd
import requests
import os

/Users/fredguth/Code/sus/sqlmesh/.env/lib/python3.11/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


Agora que já baixamos os Materiais e colocamos no BD, vamos baixar as unidades que são usadas pelos materiais que baixamos.

In [2]:
def fetch_and_save_uasgs():
    base_url = 'https://dadosabertos.compras.gov.br/modulo-uasg/1_consultarUasg'
    def save_json(status, page):
        response = requests.get(f'{base_url}?statusUasg={status}&pagina={page}').json()
        with open(f'../seeds/uasgs/status_{status}__page_{page}.json', 'w') as f:
            json.dump(response['resultado'], f)
        return response
    
    mb = master_bar([1,0])
    os.makedirs(f'../seeds/uasgs', exist_ok=True)
    for status in mb:
        response = save_json(status, 1)
        total_pages = response.get('totalPaginas', 0)
        if total_pages > 0:
            pb = progress_bar(range(2, total_pages + 1), parent=mb, leave=False)
            for page in pb:
                save_json(status, page)

In [3]:
fetch_and_save_uasgs()